# Analisis de Datos vector con R y Arcgis

Análisis basado en el tutorial de [Análisis de delitos]( https://learn.arcgis.com/es/projects/analyze-crime-using-statistics-and-the-r-arcgis-bridge/) regionalizado a la Ciudad de Medellín con datos del SISC (Sistema de Información para la Seguridad y Convivencia) publicados en el portal de mapas de Medellín. 

## 1. Instalación de R-Arcgis-Bridge y Preparación de los datos de análisis

![Ejecutar usando kernel python](https://www.python.org/static/img/python-logo.png)


In [ ]:
import arcgis, os
import arcpy
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
from IPython.display import display
# Create a GIS object, as an anonymous user for this example
gis = GIS()


In [ ]:
arcpy.env.qualifiedFieldNames = False
gdb = "c:\\esri\\r-arcgis-tools\\seguridad.gdb"
murders = "{0}\\{1}".format(gdb,"homicidios") 
robos = "{0}\\{1}".format(gdb,"")
work_dir = "c:\\esri\\crime"
work_gdb = "{0}\\analysis.gdb".format(work_dir)
cubo = "{0}\\{1}".format(work_dir,"cubo")
cubo_file = "{0}.nc".format(cubo)
hotspot_salida = "{0}\\hotspot".format(work_gdb)
hotspot_enriquecido = "{0}\\hotspot_enrich".format(work_gdb)
hotspot_suavizado =  "{0}\\hotspot_smooth".format(work_gdb)
hotspot_optimizado =  "{0}\\hotspot_optimized".format(work_gdb)

In [ ]:
if os.path.exists(cubo_file):
    os.remove(cubo_file)
if arcpy.Exists(hotspot_salida):
    arcpy.Delete_management(hotspot_salida)
if arcpy.Exists(hotspot_enriquecido):
    arcpy.Delete_management(hotspot_enriquecido)
if arcpy.Exists(hotspot_suavizado):
    arcpy.Delete_management(hotspot_suavizado)
if arcpy.Exists(hotspot_suavizado):
    arcpy.Delete_management(hotspot_suavizado)
if arcpy.Exists(hotspot_optimizado):
    arcpy.Delete_management(hotspot_optimizado)
    
if not os.path.exists(work_gdb):
    names = work_gdb.split("\\")
    idx = len(names)-1
    out_name = names[idx]
    out_folder = work_gdb.replace("\\"+out_name,"")
    arcpy.CreateFileGDB_management(out_folder,out_name)
cubo_file

In [ ]:
murders
spdf = SpatialDataFrame.from_featureclass(murders)
mapa = gis.map('Medellin')
mapa
spdf.plot(kind='map', map_widget= mapa)

In [ ]:
mapa

### 1.1 Crear cubo espacio temporal por agregación de puntos

Para más información [acá](https://pro.arcgis.com/es/pro-app/tool-reference/space-time-pattern-mining/create-space-time-cube.htm)



In [ ]:
campo_tiempo = "FECHA"
distancia = 300
distance_interval =  "{0} meters".format(distancia)


In [ ]:
arcpy.CreateSpaceTimeCube_stpm (
    murders,
    cubo,
    campo_tiempo,
    time_step_interval = "1 Months",
    time_step_alignment = "End time",
    distance_interval = distance_interval,
    aggregation_shape_type = "HEXAGON_GRID"
    )


### 1.2 Crear un Hotspot

Para mas información [acá](https://pro.arcgis.com/en/pro-app/tool-reference/space-time-pattern-mining/emerginghotspots.htm)

In [ ]:
variable_analysis = "COUNT"



In [ ]:
arcpy.EmergingHotSpotAnalysis_stpm(
    cubo,
    variable_analysis,
    hotspot_salida,
    conceptualization_of_spatial_relationships =  "FIXED_DISTANCE",
    define_global_window="ENTIRE_CUBE"
)

In [ ]:
mapa2 = gis.map('Medellin')
spdf2 = SpatialDataFrame.from_featureclass(hotspot_salida)
spdf2.plot(kind='map', map_widget=mapa2, renderer_type='u', col ="PATTERN" )
mapa2

## 2. Enriquecer los datos con ayuda de ArcGIS Online

2. Encontrar variables relacionadas para cada celda, con datos locales, para más información [aca](https://pro.arcgis.com/es/pro-app/tool-reference/analysis/enrich-layer.htm)


In [ ]:
vars = "populationtotals.totpop_cy;gender.males_cy;gender.females_cy;householdsbyincome.hinc01_cy;householdsbyincome.hinc02_cy;householdsbyincome.hinc03_cy;householdsbyincome.hinc04_cy;householdsbyincome.hinc05_cy;keyfacts.pp_cy;keyfacts.ppprm_cy;keyfacts.pppc_cy;keyfacts.ppidx_cy;spending.cs01_cy;spending.cs04_cy"


In [ ]:
arcpy.Enrich_analysis(hotspot_salida,hotspot_enriquecido,vars)

## 3. Realizar análisis estadísticos con la ayuda de R y R-ArcgisBinding

![Use Kernel R](https://www.r-project.org/Rlogo.png) 

In [ ]:
library (reshape2)
library (ggplot2)
library (ggmap)
library(sp)
library(spdep)
library(dplyr)
library(arcgisbinding)


In [ ]:
arc.check_product()

### Funciones predefinidas para cálculos

In [ ]:
# Get lower triangle of the correlation matrix
get_lower_tri<-function(cormat) {
  cormat[upper.tri(cormat)] <- NA
  return(cormat)
}
#
# Get upper triangle of the correlation matrix
get_upper_tri <- function(cormat) {
  cormat[lower.tri(cormat)] <- NA
  return(cormat)
}
#
reorder_cormat <- function(cormat) {
  # Use correlation between variables as distance
  dd <- as.dist((1-cormat) / 2)
  hc <- hclust(dd)
  cormat <- cormat [hc$order, hc$order]
}


In [ ]:
pathds <- "c:\\esri\\crime\\analysis.gdb\\hotspot_enrich"
pathdssuavizado <- "c:\\esri\\crime\\analysis.gdb\\hotspot_smooth"

Uso de R-ArcGIS-bridge para cargar datos en el workspace de R

In [ ]:
enrichdf <- arc.open(path=pathds)
enrichdf

In [ ]:
columnasDS <- c( 'OBJECTID','SUM_VALUE','populationtotals_totpop_cy','gender_males_cy','gender_females_cy','householdsbyincome_hinc01_cy', 
'householdsbyincome_hinc02_cy','householdsbyincome_hinc03_cy', 
 'householdsbyincome_hinc04_cy','householdsbyincome_hinc05_cy', 
'keyfacts_pp_cy','keyfacts_ppprm_cy','keyfacts_pppc_cy', 
'keyfacts_ppidx_cy','spending_cs01_cy','spending_cs04_cy')

In [ ]:
enrich_select_df <- arc.select(object = enrichdf, fields = columnasDS )
enrich_select_df

In [ ]:
enrich_spdf <- arc.data2sp(enrich_select_df)
enrich_spdf

In [ ]:
plot(enrich_spdf)

In [ ]:
colnames(enrich_spdf@data) <- columnasDS
colnames(enrich_spdf@data)
filtrado <- filter(enrich_spdf@data,populationtotals_totpop_cy>0)
head(filtrado)

## 3.1. Calculo de Indice de criminalidad suavizado
Estimador bayesiano de criminalidad véase [Global Empirical Bayes Estimator](https://www.rdocumentation.org/packages/spdep/versions/1.1-2/topics/EBest)


In [ ]:
n <- filtrado$SUM_VALUE
x <- filtrado$populationtotals_totpop_cy
EB <- EBest (n, x)
p <- EB$raw
b <- attr(EB, "parameters")$b
a <- attr(EB, "parameters")$a
v <- a + (b/x)
v[v < 0] <- b/x
z <- (p - b)/sqrt(v)


In [ ]:
m <- attr(EB, "parameters")$m
m
a 
b

EB


In [ ]:
z

In [ ]:
newds <- filtrado
newds$EB_Rate <- z

In [ ]:
enrich_spdf@data <- newds
arc.write(pathdssuavizado,enrich_spdf)

In [ ]:
library (ggmap)

In [ ]:
analysisColumns =c(columnasDS[2:12],"EB_Rate")
analysisColumns

### 3.2 Análisis de Correlaciones con variables relacionadas

In [ ]:
corr_sub <- newds [ analysisColumns]
cormax <- round (cor(corr_sub), 2)
upper_tri <- get_upper_tri (cormax)
melted_cormax <- melt (upper_tri, na.rm = TRUE)
cormax <- reorder_cormat (cormax)
upper_tri <- get_upper_tri (cormax)
melted_cormax <- melt (upper_tri, na.rm = TRUE)
ggheatmap <- ggplot (melted_cormax, aes (Var2, Var1, fill = value)) +
  geom_tile(color = "white") +
  scale_fill_gradient2 (low = "blue", high = "red", mid = "white", midpoint = 0, limit = c(-1,1), space = "Lab", name = "Pearson\nCorrelation") +
  theme_minimal() + # minimal theme
  theme (axis.text.x = element_text(angle = 45, vjust = 1, size = 12, hjust = 1)) +
  coord_fixed()
print (ggheatmap)
ggheatmap +
  geom_text (aes (Var2, Var1, label = value), color = "black", size = 4) +
  theme (
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    panel.grid.major = element_blank(),
    panel.border = element_blank(),
    axis.ticks = element_blank(),
    legend.justification = c (1, 0),
    legend.position = c (0.6, 0.7),
    legend.direction = "horizontal") +
  guides (fill = guide_colorbar (barwidth = 7, barheight = 1, title.position = "top", title.hjust = 0.5))

### Mejorar el Hotspot de Salida

![Ejecutar usando kernel python](https://www.python.org/static/img/python-logo.png)


In [ ]:
import arcpy
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
arcpy.env.qualifiedFieldNames = False
gis = GIS()

In [ ]:
work_dir = "c:\\esri\\crime"
work_gdb = "{0}\\analysis.gdb".format(work_dir)
hotspot_suavizado ="{0}\\hotspot_smooth".format(work_gdb)
hotspot_optimizado =  "{0}\\hotspot_optimized".format(work_gdb)
hotspot_optimizado

In [ ]:
Analysis_Field = "EB_Rate"
arcpy.OptimizedHotSpotAnalysis_stats(
    hotspot_suavizado,
    hotspot_optimizado,
    Analysis_Field = Analysis_Field
)

In [ ]:
spdf_salida = SpatialDataFrame.from_featureclass(hotspot_optimizado)


In [ ]:
mapa_salida = gis.map('Medellin')
spdf_salida.plot(kind='map', map_widget= mapa_salida, renderer_type='u', col ="Gi_Bin")
mapa_salida
